In [2]:
import numpy as np
import pandas as pd
import os
import datetime
from collections import OrderedDict
import datetime as dt
import QuantLib as ql
from scipy.stats import norm

File collection

In [328]:
daily_chains = OrderedDict()
files = sorted(os.listdir('C:\\Users\\Sean\\Downloads\\spy_data\\spy_data'))
treasury_rates = pd.read_csv("C:\\Users\\Sean\\Downloads\\daily-treasury-rates (1).csv")

for file in files:
    if file[-4:] == '.csv':
        df = pd.read_csv('C:\\Users\\Sean\\Downloads\\spy_data\\spy_data\\' + file)        
        
        # moving to datetime and making features
        df['quote_datetime'] = pd.to_datetime(df['quote_datetime'])
        df['expiration'] = pd.to_datetime(df['expiration'])
        df['quote_date'] = df['quote_datetime'][0].date()
        df['quote_date'] = pd.to_datetime(df['quote_date'])
        
        # getting only 4:00 quotes
        eod = datetime.datetime.combine(df['quote_datetime'][0].date(), datetime.time(16,0, 0))
        df = df.loc[df['quote_datetime'] == eod]
        
        # getting time to expiration and moneyness
        df['T'] = df['expiration'] - df['quote_date']
        df['T'] = df['T'].dt.days
        
        # filtering for research paper criteria
        df = df.loc[df['close']!=0]
                    
        calls = df.loc[df['option_type']=='C'][['quote_date', 'expiration', 'T','implied_volatility', 'active_underlying_price', 'strike', 'bid', 'ask', 'delta', 'gamma', 'theta', 'vega', 'rho']]
        puts = df.loc[df['option_type']=='P'][['quote_date','expiration', 'T', 'implied_volatility', 'active_underlying_price', 'strike', 'bid', 'ask', 'delta', 'gamma', 'theta', 'vega', 'rho']]
        
        daily_chains[file[-14:-4]] = {'calls':calls, 'puts':puts}

In [347]:
treasury_rates["Date"] = pd.to_datetime(treasury_rates["Date"], format="%m/%d/%Y")

index = treasury_rates[treasury_rates["Date"] == "2023-10-30"].index

print(treasury_rates["1 Mo"].iloc[index].values[0])

5.56


In [257]:
def time_to_div(day):
    """fraction of years until next dividend"""
    
    # get offset 
    divs = ["09-15", "06-16", "03-17", "12-16"]
    div_dates = sorted([dt.datetime.strptime(f'{day.year}-{date}', '%Y-%m-%d') for date in divs])
    
    # get next dividend 
    for date in div_dates:
        if day <= date:
            return (date - day).days / 365
    
    return (div_dates[0].replace(year=day.year + 1) - day).days / 365

American pricing with QuantLib

In [318]:
def price_w_ql(quote, opt_type):

    exp = quote['expiration']
    day = quote['quote_date']

    # option data
    maturity_date = ql.Date(exp.day, exp.month, exp.year)
    spot_price = quote['active_underlying_price']
    strike_price = quote['strike']
    volatility = quote['implied_volatility'] # the historical vols or implied vols
    dividend_rate =  0.0141
    if opt_type == "call":
        option_type = ql.Option.Call
    elif opt_type == "put":
        option_type = ql.Option.Put

    risk_free_rate = 0.054
    div_yield = 0.0141
    day_count = ql.Actual365Fixed()
    #calendar = ql.UnitedStates()
    calendar = ql.UnitedStates(ql.UnitedStates.NYSE)
    calculation_date = ql.Date(day.day, day.month, day.year)
    ql.Settings.instance().evaluationDate = calculation_date
    
    payoff = ql.PlainVanillaPayoff(option_type, strike_price)
    settlement = calculation_date

    am_exercise = ql.AmericanExercise(settlement, maturity_date)
    american_option = ql.VanillaOption(payoff, am_exercise)

    spot_handle = ql.QuoteHandle(
    ql.SimpleQuote(spot_price)
    )
    flat_ts = ql.YieldTermStructureHandle(
        ql.FlatForward(calculation_date, risk_free_rate, day_count)
    )
    dividend_yield = ql.YieldTermStructureHandle(
        ql.FlatForward(calculation_date, div_yield, day_count)
    )
    flat_vol_ts = ql.BlackVolTermStructureHandle(
        ql.BlackConstantVol(calculation_date, calendar, volatility, day_count)
    )
    bsm_process = ql.BlackScholesMertonProcess(spot_handle, 
                                            dividend_yield, 
                                            flat_ts, 
                                            flat_vol_ts)
    
    steps = 2500
    binomial_engine = ql.BinomialVanillaEngine(bsm_process, "crr", steps)
    american_option.setPricingEngine(binomial_engine)
    return american_option.NPV()

In [319]:
price_w_ql(daily_chains['2023-10-02']['calls'].iloc[1500], "call")

42.90115252626289

American binomial tree option pricer with discrete dividends

In [260]:
S0 = 100.0  # spot stock price
K = 100.0  # strike
T = 1.0  # maturity
r = 0.1  # risk free rate
sig = 0.2  # diffusion coefficient or volatility

In [320]:
N = 25000  # number of periods or number of time steps
payoff = "call"  # payoff

dT = float(T) / N  # Delta t
u = np.exp(sig * np.sqrt(dT))  # up factor
d = 1.0 / u  # down factor

# yield is the quarterly yield
div = {'yield': 0.003556, 'freq': 0.25, 'offset': 
        time_to_div(datetime.datetime(year=2023, month = 10, day = 2, hour = 16))}

# get in time T, when dividends occur
div_times = [div['offset']+n*div['freq'] for n in range(int((T-div['offset'])/div['freq'])+1) ]

V = np.zeros(N + 1)  # initialize the price vector
S_T = np.array([(S0 * u**j * d ** (N - j)) for j in range(N + 1)]) 

a = np.exp(r * dT)  # risk free compound return
p = (a - d) / (u - d)  # risk neutral up probability
q = 1.0 - p  # risk neutral down probability

# Determine the number of dividends paid in total
num_dividends = int((T - div['offset']) / div['freq'])
dividend_adjustment = (1 - div['yield']) ** num_dividends
S_T = S_T * dividend_adjustment

if payoff == "call":
    V[:] = np.maximum(S_T - K, 0.0)
elif payoff == "put":
    V[:] = np.maximum(K - S_T, 0.0)

for i in range(N - 1, -1, -1):

        
    V[:-1] = np.exp(-r * dT) * (p * V[1:] + q * V[:-1])  # the price vector is overwritten at each step
    
    S_T = S_T * u  # it is a tricky way to obtain the price at the previous time step

    # if we pass threshold, we can divide by (1-div)
    if div_times:
        if i * dT < div_times[-1]:
            S_T = S_T / (1 - div['yield'])
            div_times.pop(-1)
    
    if payoff == "call":
        V = np.maximum(V, S_T - K)
    elif payoff == "put":
        V = np.maximum(V, K - S_T)
            

print("American BS Tree Price: ", V[0])

American BS Tree Price:  12.514116960216668


In [404]:
def my_price(quote, opt_type, rho):
    S0 = quote['active_underlying_price'] # spot stock price
    K = quote['strike']  # strike
    T = (quote['expiration'] - quote['quote_date']).days / 365 # maturity
    index = treasury_rates[treasury_rates["Date"] == quote["quote_date"]].index
    
    # Make the risk free rate align with the treasury rates of that day
    r = treasury_rates["1 Mo"].iloc[index].values[0] / 100
    
    if rho == True:
        r += 0.01
    
    sig = quote['implied_volatility']  # volatility
    div = 0.0141
    
    
    N = 2500  # number of periods or number of time steps
    payoff = opt_type  # payoff

    dT = float(T) / N  # Delta t
    u = np.exp(sig * np.sqrt(dT))  # up factor with divident
    d = 1.0 / u  # down factor

    V = np.zeros(N + 1)  # initialize the price vector
    S_T = np.array([(S0 * u**j * d ** (N - j)) for j in range(N + 1)]) 

    a = np.exp((r - div) * dT)  # risk free compound return
    p = (a - d) / (u - d)  # risk neutral up probability
    q = 1.0 - p  # risk neutral down probability

    if payoff == "call":
        V[:] = np.maximum(S_T - K, 0.0)
    elif payoff == "put":
        V[:] = np.maximum(K - S_T, 0.0)

    for i in range(N - 1, -1, -1):

        V[:-1] = np.exp(-r * dT) * (p * V[1:] + q * V[:-1])  # the price vector is overwritten at each step
        
        S_T = S_T * u  # it is a tricky way to obtain the price at the previous time step

        if payoff == "call":
            V = np.maximum(V, S_T - K)
        elif payoff == "put":
            V = np.maximum(V, K - S_T)
                

    return V[0]

print('continuous w ql:', price_w_ql(test, "call"))
print('continuous w mine:', my_price(test, 'call', False))
print('market bid:', test['bid'])
print('market ask:', test['ask'])

continuous w ql: 0.5574854112718169
continuous w mine: 0.5608236580855579
market bid: 0.59
market ask: 0.6


In [405]:
def calculate_delta(quote, opt_type, delta_S0_percent):
    original_price = my_price(quote, opt_type, False)  # Original option price

    # Increase the stock price by a small percentage
    quote_modified = quote.copy()
    quote_modified['active_underlying_price'] *= (1 + delta_S0_percent)

    # Recalculate the option price with the increased stock price
    new_price = my_price(quote_modified, opt_type, False)

    # Calculate Delta
    delta_S0 = quote['active_underlying_price'] * delta_S0_percent
    delta = (new_price - original_price) / delta_S0

    return delta

print("calculated delta", round(calculate_delta(test, "call", 0.001), 4))
print("market delta:", test["delta"])

calculated delta 0.0845
market delta: 0.0863


In [406]:
def calculate_gamma(quote, opt_type, delta_S0_percent):
    # Calculate Delta at the original stock price
    original_delta = calculate_delta(quote, opt_type, delta_S0_percent)

    # Increase the stock price and calculate the new Delta
    quote_increased = quote.copy()
    quote_increased['active_underlying_price'] *= (1 + delta_S0_percent)
    delta_increased = calculate_delta(quote_increased, opt_type, delta_S0_percent)

    # Decrease the stock price and calculate the new Delta
    quote_decreased = quote.copy()
    quote_decreased['active_underlying_price'] *= (1 - delta_S0_percent)
    delta_decreased = calculate_delta(quote_decreased, opt_type, delta_S0_percent)

    # Calculate Gamma
    delta_S0 = quote['active_underlying_price'] * delta_S0_percent
    gamma = (delta_increased - delta_decreased) / (2 * delta_S0)

    return gamma

print("calculated gamma:", round(calculate_gamma(test, "call", 0.001), 4))
print("market gamma:", test["gamma"])

calculated gamma: 0.0099
market gamma: 0.0103


In [407]:
def calculate_vega(quote, opt_type, delta_sig):
    original_price = my_price(quote, opt_type, False)  # Original option price

    # Increase the implied volatility by a small absolute amount (e.g., 1 percentage point)
    quote_modified = quote.copy()
    quote_modified['implied_volatility'] += delta_sig

    # Recalculate the option price with the modified volatility
    new_price = my_price(quote_modified, opt_type, False)

    # Calculate Vega
    vega = ((new_price - original_price) / delta_sig) / 100

    return vega

print("calculated vega:", round(calculate_vega(test, "call", 0.01), 4))
print("market vega:", test["vega"])

calculated vega: 0.1749
market vega: 0.1795


In [408]:
from pandas.tseries.offsets import BDay

def calculate_theta(quote, opt_type, delta_t_days):
    original_price = my_price(quote, opt_type, False)  # Original option price

    # Decrease the time to expiration by one business day
    quote_modified = quote.copy()
    quote_modified['expiration'] = pd.to_datetime(quote_modified['expiration']) - BDay(delta_t_days)

    # Check if the modified expiration is still in the future
    if (quote_modified['expiration'] - pd.to_datetime(quote_modified['quote_date'])).days > 0:
        # Recalculate the option price with the adjusted time to expiration
        new_price = my_price(quote_modified, opt_type, False)

        # Calculate Theta
        theta = (new_price - original_price) / delta_t_days
    else:
        theta = 0  # If expiration has passed, theta is zero

    return theta

# Example usage
print("Calculated Theta:", round(calculate_theta(test, "call", 1), 4))
print("Market Theta:", test["theta"])


Calculated Theta: -0.0513
Market Theta: -0.0489


In [409]:
def calculate_rho(quote, opt_type, delta_r):
    original_price = my_price(quote, opt_type, False)  # Original option price
    
    # Increase the risk-free interest rate by a small absolute amount (e.g., 1 percentage point)
    quote_modified = quote.copy()
    # Recalculate the option price with the modified interest rate
    new_price = my_price(quote_modified, opt_type, True)

    # Calculate Rho
    rho = ((new_price - original_price) / delta_r) / 100

    return rho

print("calculated rho:", round(calculate_rho(test, "call", 0.01), 4))
print("market rho:", test["rho"])

calculated rho: 0.0227
market rho: 0.0233


In [410]:
test = daily_chains['2023-10-03']['calls'].iloc[700]
#test_day['calls']['T'].unique()
test

quote_date                 2023-10-03 00:00:00
expiration                 2023-10-27 00:00:00
T                                           24
implied_volatility                      0.1413
active_underlying_price                 421.68
strike                                   445.0
bid                                       0.59
ask                                        0.6
delta                                   0.0863
gamma                                   0.0103
theta                                  -0.0489
vega                                    0.1795
rho                                     0.0233
Name: 6220, dtype: object

In [411]:
from pandas.tseries.offsets import BDay

day = pd.Timestamp("2023-10-02")
delta_err = 0
gamma_err = 0
vega_err = 0
theta_err = 0
rho_err = 0

k = len(daily_chains)

for i in range(len(daily_chains)):
    quote = daily_chains[day.strftime('%Y-%m-%d')]["calls"].iloc[700]
    day += BDay(1)   
    try:
        delta_err += abs(quote["delta"] - calculate_delta(quote, "call", 0.001))
        gamma_err += abs(quote["gamma"] - calculate_gamma(quote, "call", 0.001))
        vega_err += abs(quote["vega"] - calculate_vega(quote, "call", 0.001))
        theta_err += abs(quote["theta"] - calculate_theta(quote, "call", 1))
        rho_err += abs(quote["rho"] - calculate_rho(quote, "call", 0.01))
    except IndexError:
        k -= 1
        continue
        
        
print("Average Error in Delta Calculation:", round(delta_err / k, 4) * 100, "%")
print("Average Error in Gamma Calculation:", round(gamma_err / k, 4) * 100, "%")
print("Average Error in Vega Calculation:", round(vega_err / k, 4) * 100, "%")
print("Average Error in Theta Calculation:", round(theta_err / k, 4) * 100, "%")
print("Average Error in Rho Calculation:", round(rho_err / k, 4) * 100, "%")

Average Error in Delta Calculation: 0.27999999999999997 %
Average Error in Gamma Calculation: 0.1 %
Average Error in Vega Calculation: 0.48 %
Average Error in Theta Calculation: 2.1999999999999997 %
Average Error in Rho Calculation: 0.13999999999999999 %
